In [3]:
import os
import sys
from pathlib import Path

import numpy as np
import torch

dust3r_path = str(Path(os.path.join(os.getcwd())).parent / "dust3r" / "dust3r")
sys.path.append(dust3r_path)

In [5]:
sys.path.append("../../dust3r")

In [6]:
from dust3r.cloud_opt import GlobalAlignerMode, global_aligner
from dust3r.image_pairs import make_pairs
from dust3r.inference import inference
from dust3r.model import AsymmetricCroCo3DStereo
from dust3r.utils.image import load_images

Warning, cannot find cuda-compiled version of RoPE2D, using a slow pytorch version instead


/Users/kevinlu/miniconda3/envs/annotated-dust3r/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
device = "cpu"
batch_size = 1
schedule = "cosine"
lr = 0.01
niter = 300

model_name = "naver/DUSt3R_ViTLarge_BaseDecoder_512_dpt"
# you can put the path to a local checkpoint in model_name if needed
model = AsymmetricCroCo3DStereo.from_pretrained(model_name).to(device)

In [ ]:
# load_images can take a list of images or a directory
images = load_images(["../assets/Chateau1.png", "../assets/Chateau2.png"], size=512)

assert list(images[0].keys()) == ["img", "true_shape", "idx", "instance"], images[0].keys()
assert images[0]["img"].shape == torch.Size([1, 3, 384, 512])
assert np.all(images[0]["true_shape"]) == np.all(np.array([[384, 512]]))
assert images[0]["idx"] == 0
assert images[0]["instance"] == "0"

assert list(images[1].keys()) == ["img", "true_shape", "idx", "instance"], images[1].keys()
assert images[1]["img"].shape == torch.Size([1, 3, 384, 512])
assert np.all(images[1]["true_shape"]) == np.all(np.array([[384, 512]]))
assert images[1]["idx"] == 1
assert images[1]["instance"] == "1"

>> Loading a list of 3 images
 - adding ../assets/Chateau1.png with resolution 224x224 --> 512x384
 - adding ../assets/Chateau2.png with resolution 224x224 --> 512x384
 - adding ../assets/Chateau2.png with resolution 224x224 --> 512x384
 (Found 3 images)


In [11]:
# # load_images can take a list of images or a directory
# images = load_images(["../assets/Chateau1.png", "../assets/Chateau2.png", "../assets/Chateau2.png"], size=512)

# assert list(images[0].keys()) == ["img", "true_shape", "idx", "instance"], images[
#     0
# ].keys()
# assert images[0]["img"].shape == torch.Size([1, 3, 384, 512])
# assert np.all(images[0]["true_shape"]) == np.all(np.array([[384, 512]]))
# assert images[0]["idx"] == 0
# assert images[0]["instance"] == "0"

# assert list(images[1].keys()) == ["img", "true_shape", "idx", "instance"], images[
#     1
# ].keys()
# assert images[1]["img"].shape == torch.Size([1, 3, 384, 512])
# assert np.all(images[1]["true_shape"]) == np.all(np.array([[384, 512]]))
# assert images[1]["idx"] == 1
# assert images[1]["instance"] == "1"

# assert list(images[2].keys()) == ["img", "true_shape", "idx", "instance"], images[
#     2
# ].keys()
# assert images[2]["img"].shape == torch.Size([1, 3, 384, 512])
# assert np.all(images[2]["true_shape"]) == np.all(np.array([[384, 512]]))
# assert images[2]["idx"] == 2
# assert images[2]["instance"] == "2"

In [12]:
pairs = make_pairs(images, scene_graph="complete", prefilter=None, symmetrize=True)

In [13]:
assert pairs[0] == (images[1], images[0])
assert pairs[1] == (images[0], images[1])

RuntimeError: Boolean value of Tensor with more than one value is ambiguous

In [ ]:
output = inference(pairs, model, device, batch_size=batch_size)

In [ ]:
inference

In [ ]:
# for k, v in output.items():
#     print(k)
#     if type(v) == dict:
#         for k, v in v.items():
#             try:
#                 print(f"   {k} shape: {v.shape}")
#             except Exception as e:
#                 print(f"   {k}: {v}")i

In [ ]:
assert list(output.keys()) == ["view1", "view2", "pred1", "pred2", "loss"]

In [ ]:
assert list(output["view1"].keys()) == ["img", "true_shape", "idx", "instance"]
assert output["view1"]["img"].shape == torch.Size([2, 3, 384, 512])
assert output["view1"]["true_shape"].shape == torch.Size([2, 2])
assert np.all(output["view1"]["idx"] == np.array([1, 0]))
assert output["view1"]["instance"] == ["1", "0"]

In [ ]:
assert list(output["view2"].keys()) == ["img", "true_shape", "idx", "instance"]
assert output["view2"]["img"].shape == torch.Size([2, 3, 384, 512])
assert output["view2"]["true_shape"].shape == torch.Size([2, 2])
assert np.all(output["view2"]["idx"] == np.array([0, 1]))
assert output["view2"]["instance"] == ["0", "1"]

In [ ]:
image_0_idx = images[0]["idx"]
image_1_idx = images[1]["idx"]
image_0_idx_for_view_1_output = output["view1"]["idx"].index(image_0_idx)
image_1_idx_for_view_1_output = output["view1"]["idx"].index(image_1_idx)
image_0_idx_for_view_2_output = output["view2"]["idx"].index(image_0_idx)
image_1_idx_for_view_2_output = output["view2"]["idx"].index(image_1_idx)
image_0_for_view_1_output = output["view1"]["img"][image_0_idx_for_view_1_output]
image_0_for_view_2_output = output["view2"]["img"][image_0_idx_for_view_2_output]
image_1_for_view_1_output = output["view1"]["img"][image_1_idx_for_view_1_output]
image_1_for_view_2_output = output["view2"]["img"][image_1_idx_for_view_2_output]

assert torch.all(image_0_for_view_1_output == image_0_for_view_2_output)
assert torch.all(image_1_for_view_1_output == image_1_for_view_2_output)

In [ ]:
assert list(output["pred1"].keys()) == ["pts3d", "conf"]
assert output["pred1"]["pts3d"].shape == torch.Size([2, 384, 512, 3])
assert output["pred1"]["conf"].shape == torch.Size([2, 384, 512])

In [ ]:
assert list(output["pred2"].keys()) == ["conf", "pts3d_in_other_view"]
assert output["pred2"]["pts3d_in_other_view"].shape == torch.Size([2, 384, 512, 3])
assert output["pred2"]["conf"].shape == torch.Size([2, 384, 512])

In [ ]:
assert output["loss"] == None

In [ ]:
# at this stage, you have the raw dust3r predictions
view1, pred1 = output["view1"], output["pred1"]
view2, pred2 = output["view2"], output["pred2"]
# here, view1, pred1, view2, pred2 are dicts of lists of len(2)
#  -> because we symmetrize we have (im1, im2) and (im2, im1) pairs
# in each view you have:
# an integer image identifier: view1['idx'] and view2['idx']
# the img: view1['img'] and view2['img']
# the image shape: view1['true_shape'] and view2['true_shape']
# an instance string output by the dataloader: view1['instance'] and view2['instance']
# pred1 and pred2 contains the confidence values: pred1['conf'] and pred2['conf']
# pred1 contains 3D points for view1['img'] in view1['img'] space: pred1['pts3d']
# pred2 contains 3D points for view2['img'] in view1['img'] space: pred2['pts3d_in_other_view']

# next we'll use the global_aligner to align the predictions
# depending on your task, you may be fine with the raw output and not need it
# with only two input images, you could use GlobalAlignerMode.PairViewer: it would just convert the output
# if using GlobalAlignerMode.PairViewer, no need to run compute_global_alignment
scene = global_aligner(output, device=device, mode=GlobalAlignerMode.PointCloudOptimizer)
loss = scene.compute_global_alignment(init="mst", niter=niter, schedule=schedule, lr=lr)

In [ ]:
assert len(scene.imgs) == 2
assert scene.imgs[0].shape == (384, 512, 3)
assert len(scene.get_pts3d()) == 2
assert scene.get_pts3d()[0].shape == torch.Size([384, 512, 3])
assert len(scene.get_masks()) == 2
assert scene.get_masks()[0].shape == torch.Size([384, 512])

In [ ]:
# retrieve useful values from scene:
imgs = scene.imgs
focals = scene.get_focals()
poses = scene.get_im_poses()
pts3d = scene.get_pts3d()
confidence_masks = scene.get_masks()

In [ ]:
# visualize reconstruction
scene.show(viewer="gl")

In [ ]:
# find 2D-2D matches between the two images
from dust3r.utils.geometry import find_reciprocal_matches, xy_grid

pts2d_list, pts3d_list = [], []
for i in range(2):
    conf_i = confidence_masks[i].cpu().numpy()
    pts2d_list.append(xy_grid(*imgs[i].shape[:2][::-1])[conf_i])  # imgs[i].shape[:2] = (H, W)
    pts3d_list.append(pts3d[i].detach().cpu().numpy()[conf_i])

In [ ]:
assert imgs[0].shape[:2][::-1] == (512, 384)

assert confidence_masks[0].cpu().numpy().shape == (384, 512)
assert xy_grid(*imgs[0].shape[:2][::-1]).shape == (384, 512, 2)
assert pts3d[0].detach().cpu().numpy().shape == (384, 512, 3)

# number of points selected for 2d should be the same as the mask where pixels are True
assert np.sum(confidence_masks[0].cpu().numpy()) == len(pts2d_list[0])
assert np.sum(confidence_masks[0].cpu().numpy()) == len(pts3d_list[0])

assert imgs[1].shape[:2][::-1] == (512, 384)

assert confidence_masks[1].cpu().numpy().shape == (384, 512)
assert xy_grid(*imgs[1].shape[:2][::-1]).shape == (384, 512, 2)
assert pts3d[1].detach().cpu().numpy().shape == (384, 512, 3)

# number of points selected for 2d should be the same as the mask where pixels are True
assert np.sum(confidence_masks[1].cpu().numpy()) == len(pts2d_list[1])
assert np.sum(confidence_masks[1].cpu().numpy()) == len(pts3d_list[1])

In [ ]:
reciprocal_in_P2, nn2_in_P1, num_matches = find_reciprocal_matches(*pts3d_list)
print(f"found {num_matches} matches")
matches_im1 = pts2d_list[1][reciprocal_in_P2]
matches_im0 = pts2d_list[0][nn2_in_P1][reciprocal_in_P2]

In [ ]:
# visualize a few matches
import numpy as np
from matplotlib import pyplot as pl

n_viz = 10
match_idx_to_viz = np.round(np.linspace(0, num_matches - 1, n_viz)).astype(int)
viz_matches_im0, viz_matches_im1 = (
    matches_im0[match_idx_to_viz],
    matches_im1[match_idx_to_viz],
)

H0, W0, H1, W1 = *imgs[0].shape[:2], *imgs[1].shape[:2]
img0 = np.pad(imgs[0], ((0, max(H1 - H0, 0)), (0, 0), (0, 0)), "constant", constant_values=0)
img1 = np.pad(imgs[1], ((0, max(H0 - H1, 0)), (0, 0), (0, 0)), "constant", constant_values=0)
img = np.concatenate((img0, img1), axis=1)
pl.figure()
pl.imshow(img)
cmap = pl.get_cmap("jet")
for i in range(n_viz):
    (x0, y0), (x1, y1) = viz_matches_im0[i].T, viz_matches_im1[i].T
    pl.plot(
        [x0, x1 + W0],
        [y0, y1],
        "-+",
        color=cmap(i / (n_viz - 1)),
        scalex=False,
        scaley=False,
    )
pl.show(block=True)